In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

# Structure_factor

In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BoxWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(
    intensity=1
)  # Initialize a Poisson point process
window = BoxWindow([[-50, 50], [-50, 50]])
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Poisson PointPattern

# Initialize the class StructureFactor
from structure_factor.structure_factor import StructureFactor

sf_poisson = StructureFactor(poisson_pp)

# Compute the scattering intensity on allowed wavevectors
k, s_si = sf_poisson.scattering_intensity(k_max=4)

# Visualize the result
import matplotlib.pyplot as plt
sf_poisson.plot_spectral_estimator(
    k, s_si, plot_type="all", error_bar=True, bins=30, exact_sf=poisson.structure_factor, 
    scale="log",
    label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$"
)
plt.show()

In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BoxWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BoxWindow([[-50, 50], [-50, 50]])
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Poisson PointPattern

# Initialize the class StructureFactor
from structure_factor.structure_factor import StructureFactor

sf_poisson = StructureFactor(poisson_pp)

# Generalte wavevectors 
import numpy as np

x = np.linspace(-3, 3, 100)
x = x[x != 0] # Get rid of zero
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))

# chose a taper
from structure_factor.tapers import SineTaper

p = [1, 1]
taper = SineTaper(p) # First taper of the sinusoidal tapers

# Compute the scaled tapered periodogram directly debiased
s_ddtp = sf_poisson.tapered_periodogram(k=k, taper=taper, debiased=True, direct=True)

# Visualize the result
import matplotlib.pyplot as plt

sf_poisson.plot_spectral_estimator(
    k, s_ddtp, plot_type="all", exact_sf=poisson.structure_factor, error_bar=True, 
    bins=30, label=r"$\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})$")
plt.show()


In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BoxWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BoxWindow([[-50, 50], [-50, 50]])
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Poisson PointPattern

# Initialize the class StructureFactor
from structure_factor.structure_factor import StructureFactor

sf_poisson = StructureFactor(poisson_pp)

# Generalte wavevectors 
import numpy as np

x = np.linspace(-2, 2, 80)
x = x[x != 0] # Get rid of zero
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))


# Compute the scaled multitapered periodogram undirectly debiased
s_mddtp = sf_poisson.multitapered_periodogram(k, debiased=True, direct=True, p_component_max=2) # Use the family of sine tapers (default)

# Visualize the result
import matplotlib.pyplot as plt

sf_poisson.plot_spectral_estimator(
    k, s_mddtp, plot_type="all", exact_sf=poisson.structure_factor, error_bar=True, 
    bins=30, label=r"$\widehat{S}_{\mathrm{MDDTP}}((t_j)_1^4, \mathbf{k})$")
plt.show()


In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BallWindow(center=[0,0], radius=50) # Creat a ball window
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Poisson PointPattern

# Initialize the class StructureFactor
from structure_factor.structure_factor import StructureFactor

sf_poisson = StructureFactor(poisson_pp)


# Compute Bartlett isotropic estimator
k_norm, s_bi = sf_poisson.bartlett_isotropic_estimator(n_allowed_k_norm=50) # on allowed wavenumbers

# Visualize the result
import matplotlib.pyplot as plt

sf_poisson.plot_isotropic_estimator(
    k_norm, s_bi, exact_sf=poisson.structure_factor, 
    label=r"$\widehat{S}_{\mathrm{IB}}(k)$")
plt.show()


hankel_quadrature

In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BallWindow(center=[0,0], radius=40) # Creat a ball window
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window, seed=1)  # Poisson PointPattern

# Approximate the pair correlation function
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf
poisson_pcf_fv = pcf.estimate(poisson_pp, method="fv", Kest=dict(rmax=20), fv=dict(method="c", spar=0.2))

# Interpolate/extrapolate the results
r = poisson_pcf_fv["r"]
pcf_r = poisson_pcf_fv["pcf"]
poisson_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True)

# Initialize the class StructureFactor
from structure_factor.structure_factor import StructureFactor
sf_poisson = StructureFactor(poisson_pp)

# Estimating the structure factor using Baddour Chouinard quadrature
k_norm = np.linspace(1, 10, 500) # vector of wave length
k_norm, s_hbc = sf_poisson.hankel_quadrature(poisson_pcf_fct, method="BaddourChouinard", k_norm=k_norm, r_max=20,
                                                    nb_points=1000)
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson.plot_isotropic_estimator(k_norm, s_hbc, axis=axis, error_bar=True, bins=10, 
                                           label=r"$\widehat{S}_{\mathrm{BC}}(k)$")

# pair_correlation_function

In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BallWindow(center=[0,0], radius=40) # Creat a ball window
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Poisson PointPattern

# Approximate the pair correlation function
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf
ginibre_pcf_fv = pcf.estimate(poisson_pp, method="fv", Kest=dict(rmax=20), fv=dict(method="c", spar=0.2))
ginibre_pcf_fv

#plot the results
pcf.plot(ginibre_pcf_fv, exact_pcf=poisson.pair_correlation_function, 
         figsize=(7,6), color=['grey'], style=["."])

 Interpolate/ extrapolate pcf

In [ ]:
# Generate a PointPattern in a BoxWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern

poisson = HomogeneousPoissonPointProcess(intensity=1)  # Initialize a Poisson point process
window = BallWindow(center=[0,0], radius=40) # Creat a ball window
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window, seed=1)  # Poisson PointPattern

# Approximate the pair correlation function
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf
poisson_pcf_fv = pcf.estimate(poisson_pp, method="fv", Kest=dict(rmax=20), fv=dict(method="c", spar=0.2))

# Interpolate/extrapolate the results
r = poisson_pcf_fv["r"]
pcf_r = poisson_pcf_fv["pcf"]
poisson_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True, )

# Plots the results
import matplotlib.pyplot as plt
import numpy as np
x = np.linspace(0, 50, 200)
plt.plot(x, poisson_pcf_fct(x), 'b.')
plt.plot(x, poisson.pair_correlation_function(x), 'g')
plt.show()

# Isotropic_estimator

allowed_k_norm

In [ ]:
from structure_factor.isotropic_estimator import allowed_k_norm
k = allowed_k_norm(d=2, r=20, n=6)
print(k)

bartlett_estimator

In [ ]:
# Generate a PointPattern in a BallWindow
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern
import numpy as np

poisson = HomogeneousPoissonPointProcess(intensity=1/(4*np.pi) ) # Initialize a Poisson point process
window = BallWindow(radius=20, center=[0,0,0])
poisson_points = poisson.generate_sample(window=window)  # Sample a realization
poisson_pp = PointPattern(points=poisson_points, window=window)  # Point pattern

from structure_factor.isotropic_estimator import bartlett_estimator
k_norm = np.linspace(1, 5, 3) # wavenumbers
k_norm, estimation = bartlett_estimator(poisson_pp, k_norm=k_norm)


# hyperuniformity

In [ ]:
# Load Ginibre PointPattern and restrict to BoxWindow
from structure_factor.data import load_data
from structure_factor.spatial_windows import BoxWindow
ginibre_pp = load_data.load_ginibre()
ginibre_pp_box = ginibre_pp.restrict_to_window(BoxWindow([[-35, 35], [-35, 35]]))

# Approximate the structure factor
from structure_factor.structure_factor import StructureFactor
sf_ginibre_box = StructureFactor(ginibre_pp_box)
## Wavevectors
x = np.linspace(0, 3, 80) 
x = x[x != 0] # Eliminate zero
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel())) # Wavevectors
s_ddmtp = sf_ginibre_box.multitapered_periodogram(k) # Estimate the structure factor

# Regularize the results
import structure_factor.utils as utils
from structure_factor.hyperuniformity import Hyperuniformity
k_norm = utils.norm_k(k)
hyperuniformity_test = Hyperuniformity(k_norm, s_ddmtp) # Initialize Hyperuniformity

k_norm_new, s_ddmtp_new, _ = hyperuniformity_test.bin_data(bins=40) # Regularization 

# Visualization of the results
import matplotlib.pyplot as plt
fig, axis = plt.subplots(figsize=(7,5))
axis.plot(k_norm, s_ddmtp, 'b,', label="Before regularization", rasterized=True)
axis.legend()
sf_ginibre_box.plot_isotropic_estimator(k_norm_new, s_ddmtp_new, axis=axis, color='m',
                                        exact_sf=utils.structure_factor_ginibre, 
                                        label="After regularization")
plt.show()

effective_hyperuniformity

In [ ]:
# Load Ginibre PointPattern and restrict to BoxWindow
from structure_factor.data import load_data
import numpy as np
ginibre_pp = load_data.load_ginibre()

# Approximate the structure factor
from structure_factor.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp)
k_norm, s_bi = sf_ginibre.bartlett_isotropic_estimator(n_allowed_k_norm=50) # Estimate the structure factor

# Effective hyperuniformity
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
H_ginibre, _ = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=0.2)

# Visualization of the results
import matplotlib.pyplot as plt
import structure_factor.utils as utils
fitted_line = hyperuniformity_test.fitted_line # Fitted line to s_bi
x = np.linspace(0, 1, 300)
fig, axis =plt.subplots(figsize=(7,5))
axis.plot(k_norm, s_bi, 'b', marker=".", label="Approximated structure factor")
axis.plot(x, fitted_line(x), 'r--', label= "Fitted line")
axis.plot(k_norm, utils.structure_factor_ginibre(k_norm), 'g', label=r"$S(k)$")
axis.annotate('H={}'.format(H_ginibre), xy=(0, 0), xytext=(0.01,0.1),
            arrowprops=dict(facecolor='black', shrink=0.0001))
axis.legend()
axis.set_xlabel('wavelength (k)')
axis.set_ylabel(r"Structure factor ($\mathsf{S}(k)$")
plt.show()

In [ ]:
hyperuniformity_class

In [ ]:
# Load Ginibre PointPattern and restrict to BoxWindow
from structure_factor.data import load_data
import numpy as np
ginibre_pp = load_data.load_ginibre()

# Approximate the structure factor
from structure_factor.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp)
k_norm, s_bi = sf_ginibre.bartlett_isotropic_estimator(n_allowed_k_norm=40) # Estimate the structure factor

# Hyperuniformity class
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
alpha_ginibre, _ = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.4)

# Visualization of the results
import matplotlib.pyplot as plt
import structure_factor.utils as utils
fitted_poly = hyperuniformity_test.fitted_poly # Fitted polynomial to s_bi
fig, axis =plt.subplots(figsize=(7,5))
axis.plot(k_norm, s_bi, 'b', marker=".", label="Approximated structure factor")
axis.plot(k_norm, utils.structure_factor_ginibre(k_norm), 'g', label=r"$S(k)$")
axis.plot(k_norm, fitted_poly(k_norm), 'r--', label= "Fitted line")
axis.annotate(r" $\alpha$ ={}".format(alpha_ginibre), xy=(0, 0), xytext=(0.01,0.1),
            arrowprops=dict(facecolor='black', shrink=0.0001))
axis.legend()
axis.set_xlabel('wavelength (k)')
axis.set_ylabel(r"Structure factor ($\mathsf{S}(k)$")
plt.show()

# Spatial_window

In [ ]:
from structure_factor.spatial_windows import BallWindow
import matplotlib.pyplot as plt

window = BallWindow(radius=10, center=[0, 0])
points = window.rand(n=400)
plt.plot(points[:,0], points[:,1], 'b.')
plt.show()

In [ ]:
from structure_factor.spatial_windows import BoxWindow
import matplotlib.pyplot as plt

window = BoxWindow(bounds=[[-12, 10], [-20, 7]])
points = window.rand(n=400)
plt.plot(points[:,0], points[:,1], 'b.')
plt.show()